In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-upda

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-11-05 12:11:48--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2022-11-05 12:11:49 (10.2 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)



In [ ]:
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True, )
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [ ]:
df.printSchema() 

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



**Transform**

In [67]:
# Count the number of records (rows) in the dataset.
print(df.count())


960679


In [ ]:
#drop duplicate entries
df = df.dropDuplicates()


In [ ]:
#drop null rows 
df = df.dropna()


In [ ]:
# Count the number of records (rows) in the dataset again after the above cleaning
print(df.count())


960679


In [ ]:
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"]).drop_duplicates()
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R11DJ6DZCDUQGG|   18727790|B006CHML4I|     835010224| 2014-09-11|
|R15NOHFYWK37D1|   15207072|B0013HRAXE|     210856966| 2014-03-14|
|R1GE25BMVJQ0WK|    1571808|B003UPH0DM|     818666236| 2015-05-26|
|R1Q39BDXV8FH3K|   10013398|B0085BAJR0|     119968694| 2014-05-06|
| R1QZR225XU2RU|   17063906|B005QR3SX8|      41605180| 2012-02-02|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
products = df.select(["product_id", "product_title"]).drop_duplicates()
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00C89MF4A|Weide Mens Red Th...|
|B004FD38JS|TAG Heuer Men's W...|
|B00ATUFSY4|Invicta Men's 140...|
|B009BEO81I|        Fossil Riley|
|B008B39LSA|XOXO Women's XO55...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]).drop_duplicates()

In [ ]:
vine_table.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1006OPQYUMAHS|          1|            4|          5|   N|
|R1010XUX2AIF10|          5|            0|          0|   N|
|R102DONPK6YEGN|          5|            0|          0|   N|
|R102N48HXFM9WY|          5|            0|          0|   N|
|R1060WX8UX26JV|          2|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [60]:
customers = df.select(["customer_id"]).drop_duplicates()

In [ ]:
#adding a customer_count column and setting it to 1, as the customer_id column only has unique values
#customers["customer_count"] = 1

from pyspark.sql.functions import lit

customers.withColumn("customer_count", lit(1)).show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   43765005|             1|
|   44007561|             1|
|   52805367|             1|
|   18449245|             1|
|   18106128|             1|
|   44398606|             1|
|   44609431|             1|
|    6507909|             1|
|   22307556|             1|
|   18425532|             1|
|   22150304|             1|
|    1875215|             1|
|   27053947|             1|
|   13039023|             1|
|    3690416|             1|
|   28168179|             1|
|   50282767|             1|
|   11828477|             1|
|   42288193|             1|
|   43743118|             1|
+-----------+--------------+
only showing top 20 rows



**Load**

In [ ]:
from google.colab import files
files.upload()

In [ ]:
from rds_config import password

In [ ]:
 # Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<endpoint>:5432/big_data_hwk2"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [61]:
# Write DataFrame to table

customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [62]:
# Write DataFrame to table

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [64]:
# received error: org.postgresql.util.PSQLException: ERROR: column "review_date" is of type date but expression is of type character varying.  Hint: You will need to rewrite or cast the expression.
#Need to transform the dataset to fit the tables in the schema file
from pyspark.sql.types import DateType

review_id_table = review_id_table.withColumn('review_date',df['review_date'].cast(DateType()))


In [65]:
#print schema to check 
review_id_table.printSchema() 

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [66]:
# Write DataFrame to table again

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)